3 layers, good but awful results

In [1]:
import os
import pickle as pkl
import pandas as pd
import scanpy as sc
from anndata import read_h5ad
import sklearn.metrics as met

import sys

sys.path.append("/data/yosef2/users/chenling/tabula-sapiens_old/lung_eval")

from lung_eval import *

sys.path.append("/data/yosef2/users/chenling/cellontology/")
ont_path = "/data/yosef2/users/chenling/TabulaSapiensData/ontology/"
from cellontology import CellOntolgy

from utils import (
    import_lung_data,
    get_node_to_leaves,
    get_names,
    get_to_idx_mappers,
    get_high_variable_genes,
    get_leaf_sampling_probs,
    get_classification_performance,
    predict_meta,
)

important keys

obs
- labelled (train)
- unlabelled (unlabelled examples, for semi supervised methods only)
- eval (query data, should not be useful)

general
- gt: int labels 

In [2]:
def get_train_test(mode):
    """Returns train & test batches
    
    """
    # TS -> CA
    if mode == 0:
        train_batch_indices = [
            "TSP1_10X",
            "TSP1_smartseq2",
            "TSP2_10X",
            "TSP2_smartseq2",
        ]
        test_batch_indices = ["Atlas_droplet", "Atlas_facs"]

    # TS -> Reyfman
    elif mode == 1:
        train_batch_indices = [
            "TSP1_10X",
            "TSP1_smartseq2",
            "TSP2_10X",
            "TSP2_smartseq2",
        ]
        test_batch_indices = ["Reyfman"]

    # TS -> Barga
    elif mode == 2:
        train_batch_indices = [
            "TSP1_10X",
            "TSP1_smartseq2",
            "TSP2_10X",
            "TSP2_smartseq2",
        ]
        test_batch_indices = ["Barga"]

    # CA -> Reyfman
    elif mode == 3:
        train_batch_indices = ["Atlas_droplet", "Atlas_facs"]
        test_batch_indices = ["Reyfman"]

    # CA -> Barga
    elif mode == 4:
        train_batch_indices = ["Atlas_droplet", "Atlas_facs"]
        test_batch_indices = ["Barga"]
    else:
        raise ValueError
    return "".join(train_batch_indices), "".join(test_batch_indices)

In [3]:
# %load_ext nb_black

# Imports

## Import predictions

In [4]:
# import argparse

# parser = argparse.ArgumentParser(description="Process some integers.")
# parser.add_argument("--mode", type=int, default=0)
# parser.add_argument("--prefix", type=str, default='results')
# args = parser.parse_args()
# mode = int(args.mode)
# prefix = str(args.prefix)

In [5]:
mode = 4
prefix = "results3layer"
TEST_SPLIT_SEED = 2
ON_LEAVES_ONLY = False

SAVE_DIR = "/data/yosef2/users/chenling/TabulaSapiensData/hierarchial_scANVI/scVI_TSP/experiments/lung_chenling/"
file_format = "%s_train{}_test{}_leavesonly{}_{}3.h5ad" % prefix

prefix = "results1layer"
file_format = "%s_train{}_test{}_leavesonly{}_{}_B50epochs_kl1.h5ad" % prefix


prefix = "results1layer"
file_format = "%s_train{}_test{}_leavesonly{}_{}_C50epochs_kl1.h5ad" % prefix

In [6]:
# Import anndata based on scenario `mode`
train_batch_indices, test_batch_indices = get_train_test(mode=mode)
data_file = os.path.join(
    SAVE_DIR,
    file_format.format(
        train_batch_indices, test_batch_indices, ON_LEAVES_ONLY, TEST_SPLIT_SEED
    ),
)
final_data = read_h5ad(data_file)

OSError: Unable to open file (unable to open file: name = '/data/yosef2/users/chenling/TabulaSapiensData/hierarchial_scANVI/scVI_TSP/experiments/lung_chenling/results1layer_trainAtlas_dropletAtlas_facs_testBarga_leavesonlyFalse_2_C50epochs_kl1.h5ad', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

## Load ontology info

In [ ]:
def get_targets(annd):
    """Extracts cell-type GT labels
    
    """
    celltype_dict = {}
    for x in adj.nodes:
        celltype_dict[adj.nodes[x]["name"]] = x

    ontology_id = [celltype_dict[x] for x in annd.obs["free_annotation"]]
    return nodes_to_indices[ontology_id].values


def obtain_maps():
    """Computes maps between 
     1. ids (0, 1, 2, ... 100, ..)
     2. nodes (CI:XXXXXX) ==> cellontology-based
     3. cell-type names ("B-cell")
    
    """
    all_to_leaves = get_node_to_leaves(adjm)
    leaf_node_names = list(adjm[-1].columns.values) + ["low_quality"] + ["unassigned"]
    other_node_names = list(
        all_to_leaves.index[~np.isin(all_to_leaves.index, leaf_node_names)]
    )
    id_to_label, label_to_id = get_names(all_to_leaves=all_to_leaves, adj=adj)
    nodes_to_indices, indices_to_nodes = get_to_idx_mappers(
        leaf_node_names=leaf_node_names, other_node_names=other_node_names,
    )
    return dict(
        id_to_label=id_to_label,
        label_to_id=label_to_id,
        nodes_to_indices=nodes_to_indices,
        indices_to_nodes=indices_to_nodes,
    )


In [ ]:
# Load adjacency property and matrix
adj = pkl.load(
    open(
        "/data/yosef2/users/chenling/TabulaSapiensData/ontology/ontology.lung.2.flat.pkl",
        "rb",
    )
)
adjm = adj.adjacency_matrix()

maps = obtain_maps()
id_to_label = maps["id_to_label"]
label_to_id = maps["label_to_id"]
nodes_to_indices = maps["nodes_to_indices"]
indices_to_nodes = maps["indices_to_nodes"]

In [ ]:
final_data.obs["predictions_names_META"].head()

# load scnet results

In [ ]:
(scnet_obs["scnet"] == "rand").sum()

In [ ]:
scnet_format = "results_train{}_test{}_{}new_fullgenFalse/obs.scnet.csv"
scnet_file = os.path.join(
    SAVE_DIR,
    scnet_format.format(train_batch_indices, test_batch_indices, ON_LEAVES_ONLY),
)

scnet_obs = pd.read_csv(scnet_file, index_col=1)
assert np.sum(scnet_obs.index == final_data.obs.index) == final_data.shape[0]
final_data.obs["predictions_names_scnet"] = id_to_label.reindex(
    indices_to_nodes.reindex(
        scnet_obs["scnet"]
        .replace("rand", 0)
        .astype(int)
#         convert_objects(convert_numeric=True)
    )
).values

In [ ]:
# treemaper is a DataFrame that maps any node to its ancestors
# For a given cell_type a, di correspond to:
# - a if node a is di <= depth(a)
# - the ancestor of a at depth i
# d4 corresponds to the more general cell-types, d0 to the more specific

tree_mapper = []
for idx, node in (indices_to_nodes).iteritems():
    _node = node
    path = []
    for mat in adjm[::-1]:
        if _node in mat.columns:
            anc = mat.loc[:, _node].argmax()
        else:
            anc = _node
        path.append(anc)
        _node = anc
    tree_path = {"d{}".format(i): nod for i, nod in enumerate(path)}
    tree_mapper.append(dict(node=node, **tree_path,))
tree_mapper = pd.DataFrame(tree_mapper).set_index("node").assign(orig=lambda x: x.index)
tree_mapper.head()

Create dictionary of matched cell types

In [ ]:
import obonet

obo = ont_path + 'cl.obo.txt'
f = open(obo, "r")
ontology = obonet.read_obo(f)
f.close()

match = {}
for x in ontology.nodes:
    o = nx.ancestors(ontology, x)
    p = nx.descendants(ontology, x)
    sibs = []
    parents = ontology.successors(x)
    for y in parents:
        sibs += list(ontology.successors(y))
    o = [ontology.nodes[i]['name'] for i in o]
    p = [ontology.nodes[i]['name'] for i in p]
    x = ontology.nodes[x]['name']
#     match[x] = [x] + list(o) + list(p) + sibs
    match[x] = [x] + list(o) + list(p)


# Tables

## Evaluation and formatting functions metrics

In [ ]:
def format_rows(row):
    """Format (in dollars) metrics, round them and bolder the higher one
    
    """
    max_is_best = row.pop("max_is_best")
    #     print(row)
    row = row.astype(float)
    if max_is_best:
        where_max = np.argmax(np.array(row.values))
    else:
        where_max = np.argmin(np.array(row.values))
    row = row.apply(lambda x: "{0:.3f}".format(x))
    row.iloc[where_max] = "\\mathbf{{ {} }}".format(row.iloc[where_max])
    row = row.apply(lambda x: "${}$".format(x))
    return row

# evaluation function Chenling

In [ ]:
def ConfusionMatrix(y_true, y_pred, y_ref):
    """ Compute per cell type confusion matrix and cell type proportions
    
    """
    celltypes, prop = np.unique(y_true, return_counts=True)
    ref_prop = [np.mean(y_ref == x) for x in celltypes]
    prop = prop / np.sum(prop)
    mtx = confusion_matrix(y_true, y_pred, normalize="true", labels=celltypes)
    mtx = pd.DataFrame(mtx, columns=celltypes, index=celltypes)
    return (
        pd.DataFrame(
            [celltypes, prop, ref_prop], index=["celltype", "prop", "ref_prop"]
        ).T,
        mtx,
    )

In [ ]:
def PerCelltypeAccuracy(confusion_matrix, prop, match):
    """ Compute per celltype accuracy 
    
    """
    acc1 = []
    acc2 = []
    acc_detail = {}
    for x in prop['celltype']:
        m = match[x]
        print(x)

        pred = confusion_matrix.loc[x]
        acc1.append(pred[x])
        acc2.append(pred.reindex(m).sum())
        temp1 = pd.DataFrame(pred.reindex(m).values, index=['m '+x for x in m], columns=[x])
        others = [x for x in confusion_matrix.columns[pred>0] if x not in m]
        temp2 = pd.DataFrame(pred[others].values, index=others, columns=[x])
        temp = pd.concat([temp1, temp2])
        temp.dropna(inplace=True)
        acc_detail[x] = temp
    acc = deepcopy(prop)
    acc['acc'] = acc1
    acc['hier_acc'] = acc2
    return acc_detail, acc


## stacked barplot

In [ ]:
def confusion_matrix_heatmap(mtx, save=False, res_dir=None, filename=None):
    plt.figure(figsize=(15, 12))
    sns.heatmap(mtx, linewidths=0.005, cmap="OrRd")
    plt.tight_layout()
    if save:
        plt.savefig(res_dir + filename)
    else:
        plt.show()
    plt.close()

In [ ]:
def accuracy_barplot(acc, save=False, res_dir=None, filename=None):
    acc.sort_values('prop', ascending=False, inplace=True)
    sns.set()
    plot = deepcopy(acc)
    plot.set_index('celltype', inplace=True)
    plot = plot[['acc', 'hier_acc']]
    plot['hier_acc'] = plot['hier_acc'] - plot['acc']
    plot.plot(kind='bar', stacked=True, figsize=(10,6))
    plt.legend(bbox_to_anchor=(1, 0.9))
    plt.ylabel('Prediction Accuracy')
    plt.tight_layout()
    if save:
        plt.savefig(res_dir + filename)
    else:
        plt.show()
    plt.close()



## confusion matrix heatmap

## Compute metrics

In [ ]:
model_names = [
    "kNN",
    "scANVI",
#     "scVI_scANVI",
    #     "hscANVI (leaves)",
        "hscANVI (full)",
    "SVM",
    "META",
    "scnet",
]
preds_key_fmt = "predictions_names_{}"
pred_keys = [preds_key_fmt.format(model) for model in model_names]
reference_key = "free_annotation"
ari_key = "leiden"

models_renamer = {predname: "_".join(predname.split("_")[2:]) for predname in pred_keys}
# "predictions_ids_{}"
# "predictions_nodes_{}"
# "predictions_names_{}"

In [ ]:
from copy import deepcopy

In [ ]:
# if "X_umap" not in final_data.obsm_keys():
#     sc.pp.neighbors(final_data, n_neighbors=30, n_pcs=50, use_rep="scvi_latent")
#     sc.tl.umap(final_data)
#     final_data.write_h5ad(data_file)

In [ ]:
def plot_acc(adata, y_pred, y_true, y_ari, y_ref, where_eval, where_leaves, method, plotting=False):
    plot_dir = data_file.replace(".h5ad", "")
    if not os.path.isdir(plot_dir):
        os.mkdir(plot_dir)
    # Adjusted Rand index based on reference `y_ari`
    
    adata.obs['pred'] = y_pred
    # Overall accuracy
    excluded = y_pred.isna()
    if excluded.mean()>0:
        print(excluded.mean())
    where_eval = where_eval & ~excluded
    if plotting:
        fig = sc.pl.umap(adata[where_eval], color=["pred"], return_fig=True)
        fig.savefig(plot_dir + "/test.%s.umap.pdf"%method, bbox_inches="tight")

    prop, mtx = ConfusionMatrix(y_true[where_eval], y_pred[where_eval], y_ref)
    confusion_matrix_heatmap(
        mtx, save=True, res_dir=plot_dir, filename="/test.%s.heatmap.pdf"%method
    )

    acc_detail, acc = PerCelltypeAccuracy(mtx, prop, match)
    if plotting:
        accuracy_barplot(acc.loc[acc["ref_prop"] > 0], save=True, res_dir=plot_dir, filename="/test.%s.inref.barplot.pdf"%method)
        accuracy_barplot(acc.loc[acc["ref_prop"] == 0], save=True, res_dir=plot_dir, filename="/test.%s.outref.barplot.pdf"%method)

    # Accuracy on leaves
    where_leaves_eval = where_eval & where_leaves
    #     where_leaves_eval = where_leaves

    if plotting:
        fig = sc.pl.umap(adata[where_leaves_eval], color=["pred"], return_fig=True)
        fig.savefig(plot_dir + "/leaves.%s.umap.pdf"%method, bbox_inches="tight")

    prop, mtx = ConfusionMatrix(
        y_true[where_leaves_eval], y_pred[where_leaves_eval], y_ref
    )
    confusion_matrix_heatmap(
        mtx, save=True, res_dir=plot_dir, filename="/leaves.%s.heatmap.pdf"%method
    )

    lacc_detail, lacc = PerCelltypeAccuracy(mtx, prop, match)
    if plotting:
        accuracy_barplot(lacc.loc[lacc["ref_prop"] > 0], save=True, res_dir=plot_dir, filename="/testl.%s.inref.barplot.pdf"%method)
        accuracy_barplot(lacc.loc[lacc["ref_prop"] == 0], save=True, res_dir=plot_dir, filename="/testl.%s.outref.barplot.pdf"%method)

    return acc, acc_detail, lacc, lacc_detail



In [ ]:
def evaluation_procedure(y_pred, y_true, y_ari, y_ref, where_eval, where_leaves):
    plot_dir = data_file.replace(".h5ad", "")
    if not os.path.isdir(plot_dir):
        os.mkdir(plot_dir)
    # Adjusted Rand index based on reference `y_ari`
    excluded = y_pred.isna()
    if excluded.mean()>0:
        print(excluded.mean())
    where_eval = where_eval & ~excluded
    ari = met.adjusted_rand_score(
        labels_true=y_ari[where_eval], labels_pred=y_pred[where_eval]
    )
    
    # Overall accuracy
    prop, mtx = ConfusionMatrix(y_true[where_eval], y_pred[where_eval], y_ref)
    acc_detail, acc = PerCelltypeAccuracy(mtx, prop, match)
    
    # Accuracy on leaves
    where_leaves_eval = where_eval & where_leaves
    #     where_leaves_eval = where_leaves

    prop, mtx = ConfusionMatrix(
        y_true[where_leaves_eval], y_pred[where_leaves_eval], y_ref
    )

    lacc_detail, lacc = PerCelltypeAccuracy(mtx, prop, match)

    is_out = ~np.isin(y_pred[where_eval], np.unique(y_true[where_eval]))
    poe = is_out.mean()
    res = {
        "ARI": ari,
        "w.Acc": acc["acc"].mean(),
        "w.l. Acc": lacc["acc"].mean(),
        "w.h. Acc": acc["hier_acc"].mean(),
        "w.h.l. Acc": lacc["hier_acc"].mean(),
        "POE": poe,
    }
    return res



In [ ]:
# __y_true=final_data.obs[reference_key].values
# __y_ari=final_data.obs[ari_key].values
# __y_ref=final_data.obs.loc[
# final_data.obs["labelled"].values == "True", reference_key
# ].values
# __where_eval=(final_data.obs["eval"] == "True").values
# __where_leaves=(final_data.obs["eval_leaves"] == "True").values

# __y_pred = final_data.obs[pred_keys].iloc[:, 0]

# prop, mtx = ConfusionMatrix(__y_true[__where_eval], __y_pred[__where_eval], __y_ref)
# confusion_matrix_heatmap(
#     mtx, save=True, res_dir="savedir", filename="/test.%s.heatmap.pdf"%"toto"
# )

# acc_detail, acc = PerCelltypeAccuracy(mtx, prop, match)

# for x in prop['celltype']:
#     break
# m = match[x]
# print(x)

# pred = mtx.loc[x]
# pred[x]
# pred[m].sum()

In [ ]:
table_f = (
    final_data.obs[pred_keys]
    .apply(
        evaluation_procedure,
        y_true=final_data.obs[reference_key].values,
        y_ari=final_data.obs[ari_key].values,
        y_ref=final_data.obs.loc[
            :, #final_data.obs["labelled"].values == "True", 
            reference_key
        ].values,
        where_eval=(final_data.obs["eval"] == "True").values,
        where_leaves=(final_data.obs["eval_leaves"] == "True").values,
        axis=0,
        raw=True,
    )
    .rename(columns=models_renamer)
    .loc[
    [
        "ARI",
        "POE",
        "w.Acc",
        "w.h. Acc",
    ]
]
    # Placeholder for singlecellnet, comment as soon as scn is included in the study
    .loc[:, 
         [
             "SVM", 
             "kNN", 
             "scnet",
             "hscANVI (full)",
             "scANVI", 
#              "scVI_scANVI", 
             "META",
         ],
    ]
)

## Display metrics

In [ ]:
2
(
    table_f.assign(max_is_best=[True, False, True, True]).apply(
    format_rows, axis=1
)
)
# Specify for each metric if maximum values are better
# Format the numerical values for latex copy/paste

In [ ]:
1
(
    table_f.assign(max_is_best=[True, False, True, True]).apply(
    format_rows, axis=1
)
)
# Specify for each metric if maximum values are better
# Format the numerical values for latex copy/paste

In [ ]:
4
(
    table_f.assign(max_is_best=[True, False, True, True]).apply(
    format_rows, axis=1
)
)
# Specify for each metric if maximum values are better
# Format the numerical values for latex copy/paste

In [ ]:
3
(
    table_f.assign(max_is_best=[True, False, True, True]).apply(
    format_rows, axis=1
)
)
# Specify for each metric if maximum values are better
# Format the numerical values for latex copy/paste

In [ ]:
0
(
    table_f.assign(max_is_best=[True, False, True, True]).apply(
    format_rows, axis=1
)
)
# Specify for each metric if maximum values are better
# Format the numerical values for latex copy/paste

In [ ]:
def norm_row(x):
    return x / x.max()

In [ ]:
table_f_norm = table_f.apply(norm_row, axis=1)
sns.heatmap(table_f_norm, annot=table_f, fmt=".3f")
plt.savefig(data_file.replace(".h5ad", "") + "/table_f.pdf")

In [ ]:
if mode == 0:
    print(
        "\\begin{table}[]\n"
        + "\\centering\n"
        + "\\begin{tabular}{lrrrrrr}\n"
        + "\\toprule\n"
        + "& \\textbf{%s} & \\textbf{%s} & \\textbf{%s}  & \\textbf{%s} & \\textbf{%s} & \\textbf{%s}\\\\\n"
        % ("SVM", "scVI\\_kNN", "scnet", "scANVI", "scVI\\_scANVI", "META")
   )
    print("\\textbf{TSP $\\rightarrow$ CA} & &  &  & & \\\\\n")
if mode == 1:
    print("\\hline \n")
    print("\\textbf{TSP $\\rightarrow$ Reyfman} & &  &  & & \\\\\n")
if mode == 2:
    print("\\hline \n")
    print("\\textbf{TSP $\\rightarrow$ Barga} & &  &  & & \\\\\n")
if mode == 3:
    print("\\hline \n")
    print("\\textbf{CA $\\rightarrow$ Reyfman} & &  &  & & \\\\\n")
if mode == 4:
    print("\\hline \n")
    print("\\textbf{CA $\\rightarrow$ Barga} & &  &  & & \\\\\n")



In [ ]:
for metric_name, metrics in table_f.iterrows():
    print(metric_name, " & ", " & ".join(metrics) + "\\\\")

In [ ]:
if mode == 4:
    print(
        "\\bottomrule\n"
        + "\\end{tabular}\n"
        + "\\caption {Overall classification performance for interdataset annotation (annotations subsampled to a maximum of 100 labeled examples for each cell-types using %s training}\n"
        % data_file.split("/")[-1].split("_")[0]
        + "\\label{tab}\n"
        + "\\end{table}\n"
    )

# Figures

In [ ]:
pred_keys

In [ ]:
temp = plot_acc(
    adata=final_data,
    y_pred=final_data.obs["predictions_names_scANVI"],
    y_true=final_data.obs[reference_key].values,
    y_ari=final_data.obs[ari_key].values,
    y_ref=final_data.obs.loc[
        final_data.obs["labelled"].astype(str) == "True", reference_key
    ].values,
    where_eval=(final_data.obs["eval"] == "True").values,
    where_leaves=(final_data.obs["eval_leaves"] == "True").values,
    method="_".join(x.split("_")[2:]),
)

In [ ]:
for x in pred_keys:
    plot_acc(
        adata=final_data,
        y_pred=final_data.obs[x],
        y_true=final_data.obs[reference_key].values,
        y_ari=final_data.obs[ari_key].values,
        y_ref=final_data.obs.loc[
            final_data.obs["labelled"].astype(str) == "True", reference_key
        ].values,
        where_eval=final_data.obs["eval"].astype(bool).values,
        where_leaves=final_data.obs["eval_leaves"].astype(bool).values,
        method="_".join(x.split("_")[2:]),
    )

In [ ]:
pred_keys

In [ ]:
x = "predictions_names_scANVI"
acc, acc_detail, lacc, lacc_detail = plot_acc(
    adata=final_data,
    y_pred=final_data.obs[x],
    y_true=final_data.obs[reference_key].values,
    y_ari=final_data.obs[ari_key].values,
    y_ref=final_data.obs.loc[
        final_data.obs["labelled"].astype(str) == "True", reference_key
    ].values,
    where_eval=final_data.obs["eval"].astype(bool).values,
    where_leaves=final_data.obs["eval_leaves"].astype(bool).values,
    method="_".join(x.split("_")[2:]),
)

In [ ]:
acc_detail["naive thymus-derived CD4-positive, alpha-beta T cell"]

In [ ]:
acc_detail["dendritic cell, human"]

In [ ]:
acc_detail["bronchial smooth muscle cell"]